In [21]:
import json, requests
import gpt_key
from mitaskem.src.eval.data_quality import post_pdf_to_api, extract_content_to_txt, get_var_whole_extraction, \
    count_variable_extraction, get_var_cleaning_extraction, evaluate_variable_grd_extraction, \
    get_var_whole_tool_extraction, load_arizona_concise_vars, extract_text_by_color
GPT_KEY = gpt_key.GPT_KEY
API_ROOT = "http://54.227.237.7/"
DIR="/Users/chunwei/Downloads"

### 0. Extract the whole text from the paper

In [22]:
paper_name = "ijerph-18-09027-2"
pdf_path = f'{DIR}/paper/{paper_name}.pdf'
result = post_pdf_to_api(pdf_path)
print(result)

json_extract = f'{DIR}/text/{paper_name}.json'
# Check if json_response is not None before trying to write to the file
if result is not None:
    # put the json_response into a file
    with open(json_extract, 'w') as outfile:
        json.dump(result, outfile)

# Example usage:
json_input = json_extract
paper_text_file = f'{DIR}/text/{paper_name}.txt'
extract_content_to_txt(json_input, paper_text_file)
# print the content of the txt file
with open(paper_text_file, 'r') as f:
    text = f.read()
print(text)

[{'pdf_name': 'ijerph-18-09027-2.pdf', 'dataset_id': 'ijerph-18-09027-2', 'page_num': 1, 'img_pth': '/tmp/tmpq50zt95x/ijerph-18-09027-2.pdf_1_pad', 'pdf_dims': [0.0, 0.0, 595.276, 841.89], 'bounding_box': [82.0, 107.0, 1275.0, 195.0], 'classes': ['Equation label', 'Section Header', 'Page Header', 'Body Text', 'Figure', 'Other', 'Equation', 'Reference text', 'Figure Caption', 'Table', 'Page Footer', 'Table Note', 'Table Caption', 'Abstract'], 'scores': [-5.1738767624, -8.3762178421, -10.0131721497, -10.1932468414, -11.0821018219, -11.1463050842, -11.2431697845, -13.1342964172, -15.042175293, -15.1790990829, -15.9822711945, -18.4401931763, -18.7492847443, -19.0146560669], 'content': '', 'postprocess_cls': 'Page Header', 'postprocess_score': 0.9985097051, 'detect_cls': 'Equation label', 'detect_score': -5.1738767624}, {'pdf_name': 'ijerph-18-09027-2.pdf', 'dataset_id': 'ijerph-18-09027-2', 'page_num': 1, 'img_pth': '/tmp/tmpq50zt95x/ijerph-18-09027-2.pdf_1_pad', 'pdf_dims': [0.0, 0.0, 595

### 1. Extract the variables from the plain text with GPT4T

In [23]:
pure_gpt = get_var_whole_extraction(text, GPT_KEY)
print(pure_gpt)

Using GPT model:  gpt-4-1106-preview
β | expected number of people an infected person infects per day | None
γ | proportion of recovery per day | 1/11
δ | incubation period | 1/5
α | fatality rate due to the infection | 0.000064
ρ | inverse of the average number of days for an infected person to die if he does not recover | 1/9
κ | arbitrary constant used in the infection rate model | None
βs | infection rate before full compliance masking enforcement | 1
βc | infection rate after full compliance masking enforcement | 0.4
t0 | number of days after the first case where masking wearing is enforced | None
βnc | infection rate when there is noncompliance | 0.5
t1 | time after the first case when onset of noncompliance is triggered | 154
k1, k2 | arbitrary constants tuning the gradients in the infection rate transition | None
 | rate at which a recovered person becomes susceptible again | None


In [24]:
# count how many variables are extracted
pure_count = count_variable_extraction(pure_gpt)
print("number of variable in mit extraction: ", pure_count)

number of variable in mit extraction:  13


### 2.1 Extract the variables from the plain text with SKEMA tool

In [25]:
# evaluate the whole text extraction with Arizona tool
API_ROOT = "https://api.askem.lum.ai/"
paper_pdf = pdf_path
with open(paper_pdf, 'rb') as f:
    params = {
        "annotate_skema": True,
        "annotate_mit": False
    }
    files = [("pdfs", (paper_pdf, f))]
    response = requests.post(API_ROOT + "text-reading/integrated-pdf-extractions", params=params, files=files)
    json_str = response.text

tool_response_json = json.loads(json_str)
# create a dummy json file for tool extraction
tool_json_file = '/tmp/mit.json'
with open(tool_json_file, 'w') as outfile:
    json.dump(tool_response_json, outfile)
tool_concise = tool_json_file.replace(".json", "-concise.txt")
load_arizona_concise_vars(tool_json_file, tool_concise)
tool_text = open(tool_concise).read()
print(tool_text)

count_tool = count_variable_extraction(tool_text)
print("number of variable in Arizona tool extraction: ", count_tool)

E:-1621000196, potential prevention of up: 
R:1863362492, beta(t): 
R:-2131818727, beta(t): beta
R:-136675206, beta(t): beta
E:-375195508, betac: 
E:-227734974, betas: 
R:2011356353, beta(t): beta
E:1157924531, beta(t): beta
R:526915523, beta(t): beta
R:-328739983, I(t): infection rate
E:1248386289, t0: number of days after the first case
R:46512816, policy of compulsory mask wearing: 
R:-407044703, example of such logistic function: 
R:-1111531285, 47%: 
E:-445420568, t1: 
E:-1296333892, betanc: 
R:-2064009007, k1: arbitrary constants
E:-262180774, case: 
E:307660388, case of a (b) 50 days delay: 
E:-306701002, case of a (b) 50 days delay: 
E:-1711474770, case of a (b) 50 days delay: 
R:-1134010888, a: be enforced
E:-1856653836, maximum infected value: 
E:-223017254, maximum infected value: 
E:-1463565746, maximum infected value: 
R:-823594056, case of (a) 0 days of delay: 
E:-232718024, maximum infected value: 
E:-429168824, maximum infected value: 
E:1232232041, maximum infected val

### 2.2 Extract the variables from the plain text with SKEMA tool

In [26]:
# test the whole text extraction with Arizona tool support
tool_gpt = get_var_whole_tool_extraction(text, tool_text, GPT_KEY)
print(tool_gpt)
count_tool_gpt = count_variable_extraction(tool_gpt)
print("number of variable in GPT-tool extraction: ", count_tool_gpt)

Using GPT model:  gpt-4-1106-preview
N | total population | None
S(t) | number of people susceptible on day t | None
E(t) | number of people exposed on day t | None
I(t) | number of people infected on day t | None
R(t) | number of people recovered on day t | None
D(t) | number of people dead on day t | None
β | expected number of people an infected person infects per day | None
γ | proportion of recovery per day | 1/11
δ | incubation period | 1/5
α | fatality rate due to the infection | 0.000064
ρ | inverse of the average number of days for an infected person to die if he does not recover | 1/9
βs | infection rate before full compliance masking enforcement | 1
βc | infection rate after full compliance masking enforcement | 0.4
βnc | infection rate after noncompliance begins | 0.5
t0 | number of days after the first case where masking wearing is enforced | 89
t1 | onset of noncompliance triggered by an event | 154
k1 | arbitrary constant for gradient tuning | None
k2 | arbitrary constan

### 3.1 Evaluate the extraction with ground truth
Get the ground truth from the json file which is converted from the the annotated papers. The ground truth is the text with yellow color.

In [27]:

# get the ground truth
ground_truth = f'{DIR}/json/{paper_name}.json'
grd_text = ground_truth.replace(".json", "-concise.txt")
extract_text_by_color(ground_truth, grd_text, "#ffd100")
grd_text = open(grd_text).read()
print(grd_text)

count_grd = count_variable_extraction(grd_text)
print("number of variable in ground truth: ", count_grd)

S(t), E(t), I(t), R(t) and D(t), are the number of people susceptible, exposed, infected, recovered and dead on day t
β is the expected number of people an infected person infects per day,
δ is the incubation period
γ is the proportion of recovery per day,
ρ is the inverse of the average number of days for an infected person to die if he does not recover
α is the fatality rate due to the infection
t0 is the number of days after the first case where masking wearing is enforced
infection rate, βnc

number of variable in ground truth:  8


### 3.2 Ground truth cleaning to remove duplicate variable extractions

In [28]:
grd_clean = get_var_cleaning_extraction(grd_text, GPT_KEY)
print(grd_clean)
count_grd_clean = count_variable_extraction(grd_clean)
print("number of variable in cleaning ground truth: ", count_grd_clean)

Using GPT model:  gpt-4-1106-preview
S(t): the number of people susceptible on day t
E(t): the number of people exposed on day t
I(t): the number of people infected on day t
R(t): the number of people recovered on day t
D(t): the number of people dead on day t
β: the expected number of people an infected person infects per day
δ: the incubation period
γ: the proportion of recovery per day
ρ: the inverse of the average number of days for an infected person to die if he does not recover
α: the fatality rate due to the infection
t0: the number of days after the first case where masking wearing is enforced
number of variable in cleaning ground truth:  11


### 4.1 Evaluate the pure GPT extraction with ground truth

In [29]:
# Evaluate the MIT extraction with clean ground truth
pure_match = evaluate_variable_grd_extraction(pure_gpt, grd_clean, GPT_KEY)
print(pure_match)
count_pure_match = count_variable_extraction(pure_match)
print("Number of variable in pure GPT match: ", count_pure_match)
print("GPT precision: ", count_pure_match / pure_count)
print("GPT recall: ", count_pure_match / count_grd_clean)

Using GPT model:  gpt-4-1106-preview
β: expected number of people an infected person infects per day
γ: proportion of recovery per day
δ: incubation period
α: fatality rate due to the infection
ρ: inverse of the average number of days for an infected person to die if he does not recover
t0: number of days after the first case where masking wearing is enforced
Number of variable in pure GPT match:  6
GPT precision:  0.46153846153846156
GPT recall:  0.5454545454545454


### 4.2 Evaluate the GPT+Tool extraction with ground truth

In [30]:
# Evaluate the MIT tool extraction with clean ground truth
tool_match = evaluate_variable_grd_extraction(tool_gpt, grd_clean, GPT_KEY)
print(tool_match)
count_tool_match = count_variable_extraction(tool_match)
print("Number of variable in GPT+Tool match: ", count_tool_match)
print("GPT+Tool precision: ", count_tool_match / count_tool_gpt)
print("GPT+Tool recall: ", count_tool_match / count_grd_clean)

Using GPT model:  gpt-4-1106-preview
S(t): number of people susceptible on day t
E(t): number of people exposed on day t
I(t): number of people infected on day t
R(t): number of people recovered on day t
D(t): number of people dead on day t
β: expected number of people an infected person infects per day
γ: proportion of recovery per day
δ: incubation period
α: fatality rate due to the infection
ρ: inverse of the average number of days for an infected person to die if he does not recover
t0: number of days after the first case where masking wearing is enforced
Number of variable in GPT+Tool match:  11
GPT+Tool precision:  0.5789473684210527
GPT+Tool recall:  1.0
